<a href="https://colab.research.google.com/github/faizuddin/IBB31103/blob/main/timeseries_univariate_ws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Timeseries Data Forecasting Using RNN

In [ ]:
# load and plot dataset
import pandas as pd

# load dataset
def parser(x):
	return pd.datetime.strptime("190"+x, "%Y-%m")
series = pd.read_csv("https://raw.githubusercontent.com/faizuddin/workshop/main/shampoo.csv", header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [ ]:
from matplotlib import pyplot

# summarize first few rows
print(series.head())

# line plot
series.plot()
pyplot.show()

## LSTM Data Preparation

Before we can fit an LSTM model to the dataset, we must transform the data.

This section is broken down into three steps:

1. Transform the time series into a supervised learning problem
2. Transform the time series data so that it is stationary.
3. Transform the observations to have a specific scale.

### Transform Time Series to Supervised Learning

The LSTM model in Keras assumes that your data is divided into input (`X`) and output (`y`) components.

For a time series problem, we can achieve this by using the observation from the last time step (`t-1`) as the input and the observation at the current time step (`t`) as the output.

We can achieve this using the `shift()` function in Pandas that will push all values in a series down by a specified number places. We require a shift of `1` place, which will become the input variables. The time series as it stands will be the output variables.

We can then concatenate these two series together to create a DataFrame ready for supervised learning. The pushed-down series will have a new position at the top with no value. A `NaN` (not a number) value will be used in this position. We will replace these `NaN` values with `0` values, which the LSTM model will have to learn as *“the start of the series”* or *“I have no data here,”* as a month with zero sales on this dataset has not been observed.

The code below defines a helper function to do this called `timeseries_to_supervised()`. It takes a NumPy array of the raw time series data and a lag or number of shifted series to create and use as inputs.

In [ ]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = pd.DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = pd.concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

We can test this function with our loaded Shampoo Sales dataset and convert it into a supervised learning problem.

The code prints the first 5 rows of the new supervised learning problem.

In [ ]:
# transform to supervised learning
X = series.values
supervised = timeseries_to_supervised(X, 1)
print(supervised.head())

### Transform Time Series to Stationary

The Shampoo Sales dataset is not **stationary**. This means that there is a structure in the data that is dependent on the time. Specifically, there is an increasing trend in the data.

Stationary data is easier to model and will very likely result in more skillful forecasts.

The trend can be removed from the observations, then added back to forecasts later to return the prediction to the original scale and calculate a comparable error score.

A standard way to remove a trend is by differencing the data. That is the observation from the previous time step (`t-1`) is subtracted from the current observation (`t`). This removes the trend and we are left with a difference series, or the changes to the observations from one time step to the next.

We can achieve this automatically using the `diff()` function in pandas. Alternatively, we can get finer grained control and write our own function to do this, which is preferred for its flexibility in this case.

Below is a function called `difference()` that calculates a differenced series. Note that the first observation in the series is skipped as there is no prior observation with which to calculate a differenced value.

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
  diff = list()
  for i in range(interval, len(dataset)):
    value = dataset[i] - dataset[i - interval]
    diff.append(value)
  return pd.Series(diff)

We also need to invert this process in order to take forecasts made on the differenced series back into their original scale.

The function below, called `inverse_difference()`, inverts this operation.

In [ ]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

We can test out these functions by differencing the whole series, then returning it to the original scale, as follows:

In [ ]:
# transform to be stationary
differenced = difference(series, 1)
print("Difference\n\n", differenced.head())

# invert transform
inverted = list()
for i in range(len(differenced)):
	value = inverse_difference(series, differenced[i], len(series)-i)
	inverted.append(value)
 
inverted = pd.Series(inverted)
print("Inverse:\n\n", inverted.head())

### Transform Time Series to Scale
Like other neural networks, LSTMs expect data to be within the scale of the activation function used by the network.

The default activation function for LSTMs is the *hyperbolic tangent* (`tanh`), which outputs values between `-1` and `1`. This is the preferred range for the time series data.

To make the experiment fair, the scaling coefficients (min and max) values must be calculated on the training dataset and applied to scale the test dataset and any forecasts. This is to avoid contaminating the experiment with knowledge from the test dataset, which might give the model a small edge.

We can transform the dataset to the range `[-1, 1]` using the `MinMaxScaler` class. Like other `scikit-learn` transform classes, it requires data provided in a matrix format with rows and columns. Therefore, we must reshape our `NumPy` arrays before transforming.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = series.values
X = X.reshape(len(X), 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(X)
scaled_X = scaler.transform(X)

Again, we must invert the scale on forecasts to return the values back to the original scale so that the results can be interpreted and a comparable error score can be calculated.

In [ ]:
# invert transform
inverted_X = scaler.inverse_transform(scaled_X)

The example below transforms the scale of the Shampoo Sales data.

In [ ]:
# transform scale
X = series.values
X = X.reshape(len(X), 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(X)
scaled_X = scaler.transform(X)
scaled_series = pd.Series(scaled_X[:, 0])
print("Scaled\n\n", scaled_series.head())

# invert transform
inverted_X = scaler.inverse_transform(scaled_X)
inverted_series = pd.Series(inverted_X[:, 0])
print("Inversed\n\n", inverted_series.head())

## LSTM Model Development

The Long Short-Term Memory network (LSTM) is a type of Recurrent Neural Network (RNN).

A benefit of this type of network is that it can learn and remember over long sequences and does not rely on a pre-specified window lagged observation as input.

In Keras, this is referred to as *stateful*, and involves setting the *stateful* argument to “True” when defining an LSTM layer.

By default, an LSTM layer in Keras maintains state between data within one batch. A batch of data is a fixed-sized number of rows from the training dataset that defines how many patterns to process before updating the weights of the network. State in the LSTM layer between batches is cleared by default, therefore we must make the LSTM stateful. This gives us fine-grained control over when state of the LSTM layer is cleared, by calling the reset_states() function.

The LSTM layer expects input to be in a matrix with the dimensions: `[samples, time steps, features]`.

* Samples: These are independent observations from the domain, typically rows of data.
* Time steps: These are separate time steps of a given variable for a given observation.
* Features: These are separate measures observed at the time of observation.

We have some flexibility in how the Shampoo Sales dataset is framed for the network. We will keep it simple and frame the problem as each time step in the original sequence is one separate sample, with one timestep and one feature.

Given that the training dataset is defined as `X` inputs and `y` outputs, it must be reshaped into the Samples/TimeSteps/Features format:

```
X, y = train[:, 0:-1], train[:, -1]
X = X.reshape(X.shape[0], 1, X.shape[1])
```

### Input Shape, Batch Size and Neuron
The shape of the input data must be specified in the LSTM layer using the `batch_input_shape` argument as a tuple that specifies the expected number of observations to read each batch, the number of time steps, and the number of features.

The batch size is often much smaller than the total number of samples. It, along with the number of epochs, defines how quickly the network learns the data (how often the weights are updated).

The final import parameter in defining the LSTM layer is the number of neurons, also called the number of memory units or blocks. This is a reasonably simple problem and a number between 1 and 5 should be sufficient.

The line below creates a single LSTM hidden layer that also specifies the expectations of the input layer via the `batch_input_shape` argument.

```
from keras.layers import LSTM

layer = LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True)
```



The network requires a single neuron in the output layer with a linear activation to predict the number of shampoo sales at the next time step.

Once the network is specified, it must be compiled into an efficient symbolic representation using a backend mathematical library, in our case we are using such as TensorFlow.

In compiling the network, we must specify a loss function and optimization algorithm. We will use `mean_squared_error` as the loss function as it closely matches RMSE that we will are interested in, and the efficient `ADAM` optimization algorithm.

Using the Sequential Keras API to define the network, the below snippet creates and compiles the network.

```
model = Sequential()
model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
```

Once compiled, it can be fit to the training data. Because the network is stateful, we must control when the internal state is reset. Therefore, we must manually manage the training process one epoch at a time across the desired number of epochs.

By default, the samples within an epoch are shuffled prior to being exposed to the network. Again, this is undesirable for the LSTM because we want the network to build up state as it learns across the sequence of observations. We can disable the shuffling of samples by setting “shuffle” to “False“.

Also by default, the network reports a lot of debug information about the learning progress and skill of the model at the end of each epoch. We can disable this by setting the “verbose” argument to the level of “0“.

We can then reset the internal state at the end of the training epoch, ready for the next training iteration.

Below is a loop that manually fits the network to the training data:

```
for i in range(nb_epoch):
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
	model.reset_states()
```

Putting this all together, we can define a function called `fit_lstm()` that trains and returns an LSTM model. As arguments, it takes the training dataset in a supervised learning format, a batch size, a number of epochs, and a number of neurons.

```
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model
```

The batch_size must be set to `1`. This is because it must be a factor of the size of the training and test datasets.

The `predict()` function on the model is also constrained by the batch size; there it must be set to `1` because we are interested in making one-step forecasts on the test data.

We will not tune the network parameters in this tutorial; instead we will use the following configuration, found with a little trial and error:

* Batch Size: 1
* Epochs: 3000
* Neurons: 4

*Consider trying 1500 epochs and 1 neuron, the performance may be better?*

## LSTM Forecast

Once the LSTM model is fit to the training data, it can be used to make forecasts.

Again, we have some flexibility. We can decide to fit the model once on all of the training data, then predict each new time step one at a time from the test data (we’ll call this the fixed approach), or we can re-fit the model or update the model each time step of the test data as new observations from the test data are made available (we’ll call this the dynamic approach).

In this tutorial, we will go with the fixed approach for its simplicity, although, we would expect the dynamic approach to result in better model skill.

To make a forecast, we can call the `predict()` function on the model. This requires a 3D NumPy array input as an argument. In this case, it will be an array of one value, the observation at the previous time step.

The `predict()` function returns an array of predictions, one for each input row provided. Because we are providing a single input, the output will be a 2D NumPy array with one value.

We can capture this behavior in a function named `forecast()` listed below. Given a fit model, a batch-size used when fitting the model (e.g. 1), and a row from the test data, the function will separate out the input data from the test row, reshape it, and return the prediction as a single floating point value.

```
def forecast(model, batch_size, row):
	X = row[0:-1]
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]
```

During training, the internal state is reset after each epoch. While forecasting, we will not want to reset the internal state between forecasts. In fact, we would like the model to build up state as we forecast each time step in the test dataset.

This raises the question as to what would be a good initial state for the network prior to forecasting the test dataset.

In this tutorial, we will seed the state by making a prediction on all samples in the training dataset. In theory, the internal state should be set up ready to forecast the next time step.

We now have all of the pieces to fit an LSTM Network model for the Shampoo Sales dataset and evaluate its performance.

## Full LTSM Pipeline

This will involve drawing together all of the elements from the prior sections. There are a lot of them, so let’s review:

1. Load the dataset from CSV file.
3. Transforming the data to be stationary.
4. Transforming the data to a supervised learning problem.
5. Transforming the data so that it has the scale -1 to 1.
6. Fitting a stateful LSTM network model to the training data.
7. Evaluating the static LSTM model on the test data.
8. Report the performance of the forecasts.


### Load Dataset

In [ ]:
import pandas as pd
# date-time parsing function for loading the dataset
def parser(x):
	return pd.datetime.strptime('190'+x, '%Y-%m')
 
 # load dataset
series = pd.read_csv("https://raw.githubusercontent.com/faizuddin/workshop/main/shampoo.csv", header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

### Trending to stationary

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return pd.Series(diff)
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]


# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, 1)

### Timeseries to supervised learning

In [ ]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = pd.DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = pd.concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

### Training-Test split

In [ ]:
# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]

### Scale dataset

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled
  
# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

### Train LTSM

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model

# fit the model
lstm_model = fit_lstm(train_scaled, 1, 3000, 4)

# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

### Evaluating the static LSTM model on the test data.

In [ ]:
import numpy as np

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = np.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

# walk-forward validation on the test data
predictions = list()
for i in range(len(test_scaled)):
	# make one-step forecast
	X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
	yhat = forecast_lstm(lstm_model, 1, X)
	
  # invert scaling
	yhat = invert_scale(scaler, X, yhat)
	
  # invert differencing
	yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
	
  # store forecast
	predictions.append(yhat)
	expected = raw_values[len(train) + i + 1]
	print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

### Performance of the forecasts.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot

# report performance
rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
print('Test RMSE: %.3f' % rmse)

# line plot of observed vs predicted
pyplot.plot(raw_values[-12:])
pyplot.plot(predictions)

## Develop a Robust Result

A difficulty with neural networks is that they give different results with different starting conditions.

One approach might be to fix the random number seed used by Keras to ensure the results are reproducible. Another approach would be to control for the random initial conditions using a different experimental setup.

We can repeat the experiment from the previous section multiple times, then take the average RMSE as an indication of how well the configuration would be expected to perform on unseen data on average.

This is often called multiple repeats or multiple restarts.

We can wrap the model fitting and walk-forward validation in a loop of fixed number of repeats. Each iteration the RMSE of the run can be recorded. We can then summarise the distribution of RMSE scores.

In [ ]:
# repeat experiment
repeats = 10

error_scores = list()
for r in range(repeats):
  # fit the model
  lstm_model = fit_lstm(train_scaled, 1, 3000, 4)
  # forecast the entire training dataset to build up state for forecasting
  train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
  lstm_model.predict(train_reshaped, batch_size=1)
	
  # walk-forward validation on the test data
  predictions = list()
  
  for i in range(len(test_scaled)):
    # make one-step forecast
    X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
    yhat = forecast_lstm(lstm_model, 1, X)
    # invert scaling
    yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    # store forecast
    predictions.append(yhat)
	
  # report performance
  rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
  print('%d) Test RMSE: %.3f' % (r+1, rmse))
  error_scores.append(rmse)

### Visualise performance summary

Running the example prints the RMSE score each repeat. The end of the run provides summary statistics of the collected RMSE scores.

**Note**: Your results may vary given the stochastic nature of the algorithm or evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average outcome.

This is a very useful result as it suggests the result reported was probably a statistical fluke. The experiment suggests that the model is probably about as good as the persistence model on average, if not slightly worse.

This indicates that, at the very least, further model tuning is required.

A box and whisker plot captures the middle of the data as well as the extents and outlier results.

In [ ]:
# Summarise results
results = pd.DataFrame()
results['rmse'] = error_scores
print(results.describe())
results.boxplot()